<a href="https://colab.research.google.com/github/jawadshaik73/Rag-pipeline/blob/main/chat%20with%20pdf%20using%20rag%20pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests PyMuPDF sentence-transformers faiss-cpu transformers torch


In [ ]:
import fitz
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [ ]:
def chunk_text(text, chunk_size=300):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

In [ ]:
def create_faiss_index(chunks, model_name="all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks, convert_to_tensor=True).cpu().numpy()

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index, model

In [ ]:
def search_similar_chunks(query, index, model, chunks, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]

In [ ]:
def generate_response(query, retrieved_chunks, model, tokenizer, max_new_tokens=100, context_limit=1024):
    context = "\n".join(retrieved_chunks)
    if len(context) > context_limit:
        context = context[:context_limit]

    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"

    inputs = tokenizer.encode(prompt, return_tensors="pt")
    inputs = inputs.to(model.device)

    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def main_pipeline():
    pdf_path = "/DATA MINING UNIT-1 NOTES.pdf"
    content = extract_text_from_pdf(pdf_path)
    if not content:
        print("No content retrieved from the PDF. Exiting.")
        return
    chunks = chunk_text(content)
    print(f"Extracted {len(chunks)} chunks from the PDF content.")
    index, embedding_model = create_faiss_index(chunks)
    print("Loading GPT-2 model...")
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    llm_model = GPT2LMHeadModel.from_pretrained("gpt2")
    llm_model = llm_model.to("cuda" if torch.cuda.is_available() else "cpu")
    print("System is ready. Type your questions below!")
    while True:
        user_query = input("\nEnter your question (or 'exit' to quit): ").strip()
        if user_query.lower() == "exit":
            print("Exiting the chat. Goodbye!")
            break
        if not user_query:
            print("Please enter a valid question.")
            continue
        retrieved_chunks = search_similar_chunks(user_query, index, embedding_model, chunks)
        if not retrieved_chunks:
            print("No relevant information found.")
            continue
        response = generate_response(user_query, retrieved_chunks, llm_model, tokenizer)
        print("\nResponse:")
        print(response)
if __name__ == "__main__":
    main_pipeline()

Extracted 15 chunks from the PDF content.
Loading GPT-2 model...
System is ready. Type your questions below!

Response:
Context: Faculty: Mr. D. Krishna, Associate Professor CSE Dept DATA MINING UNIT-I Introduction to Data Mining: Data mining is the process of discovering patterns in large data sets involving methods at the intersection of machine learning, statistics, and database systems. The information or knowledge extracted so can be used for any of the following applications:  Market Analysis  Fraud Detection  Customer Retention  Production Control  Science Exploration Data Mining Applications: Data mining is highly useful in the following domains:  Market Analysis and Management  Corporate Analysis & Risk Management  Fraud Detection Apart from these, data mining can also be used in the areas of production control, customer retention, science exploration, sports, astrology, and Internet Web Surf- Aid. Knowledge discovery in databases (KDD): Knowledge discovery in database